In [1]:
import numpy as np
import keras
from keras.layers import Input, Dense, Activation, BatchNormalization, Reshape, UpSampling2D, Conv2D, MaxPool2D, Flatten

class Generator(object):
    def __init__(self, latent_dim):
        generator_input = keras.Input(shape=(latent_dim,))
        x = Dense(1024)(generator_input)
        x = Activation('tanh')(x)
        x = Dense(128*7*7)(x)
        x = BatchNormalization()(x)
        x = Activation('tanh')(x)
        x = Reshape((7, 7, 128))(x)
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(64, 5, padding='same')(x)
        x = Activation('tanh')(x)
        x = UpSampling2D(size=(2, 2))(x)
        x = Conv2D(1, 5, padding='same')(x)
        x = Activation('tanh')(x)
        
        self.generator = keras.models.Model(generator_input, x)
    def get_model(self):
        return self.generator


class Discriminator(object):
    def __init__(self, height, width, channels):
        discriminator_input = Input(shape=(height, width, channels))
        x = Conv2D(64, 5, padding='same')(discriminator_input)
        x = Activation('tanh')(x)
        x = MaxPool2D()(x)
        x = Conv2D(128, 5)(x)
        x = Activation('tanh')(x)
        x = MaxPool2D()(x)
        x = Flatten()(x)
        x = Dense(1024)(x)
        x = Activation('tanh')(x)
        x = Dense(1, activation='sigmoid')(x)
        self.discriminator = keras.models.Model(discriminator_input, x)

    def get_model(self):
        return self.discriminator
    



class DCGAN(object):
    def __init__(self, latent_dim, height, width, channels):
        # set generator
        self._latent_dim = latent_dim
        g = Generator(latent_dim)
        self._generator = g.get_model()
        
        
        
        # set discriminator
        d1 = Discriminator(height, width, channels)
        self._discriminator1 = d1.get_model()
        
        # set discriminator
        d2 = Discriminator(height, width, channels)
        self._discriminator2 = d2.get_model()
        
        
        # compile discriminator
        discriminator_optimizer = keras.optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        
        
        self._discriminator1.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')
        self._discriminator2.compile(optimizer=discriminator_optimizer, loss='binary_crossentropy')
        
        # disable training when combined with generator
        self._discriminator1.trainable = False
        # disable training when combined with generator
        self._discriminator2.trainable = False
        
        # set DCGAN
        dcgan_input = keras.Input(shape=(latent_dim,))
        dcgan_output1 = self._discriminator1(self._generator(dcgan_input))
        dcgan_output2 = self._discriminator2(self._generator(dcgan_input))
        self._dcgan = keras.models.Model(dcgan_input, [dcgan_output1,dcgan_output2])
        
        # compile DCGAN
        dcgan_optimizer = keras.optimizers.SGD(lr=0.0005, momentum=0.9, nesterov=True)
        self._dcgan.compile(optimizer=dcgan_optimizer, loss='binary_crossentropy')

    def train(self, real_images, batch_size):
        # Train so discriminator can detect fake
        random_latent_vectors = np.random.normal(size=(batch_size, self._latent_dim))
        generated_images = self._generator.predict(random_latent_vectors)
        labels = np.ones((batch_size, 1))
        labels += 0.05 * np.random.random(labels.shape)
        d1_loss1 = self._discriminator1.train_on_batch(generated_images, labels)
        d2_loss1 = self._discriminator2.train_on_batch(generated_images, labels)
        
        
        
        # Train so discriminator can detect real
        labels = np.zeros((batch_size, 1))
        labels += 0.05 * np.random.random(labels.shape)
        d1_loss2 = self._discriminator1.train_on_batch(real_images, labels)
        d2_loss2 = self._discriminator1.train_on_batch(real_images, labels)
        d1_loss = (d1_loss1 + d1_loss2)/2.0
        d2_loss = (d2_loss1 + d2_loss2)/2.0
        
        # Train so generator can fool discriminator
        random_latent_vectors = np.random.normal(size=(batch_size, self._latent_dim))
        misleading_targets = np.zeros((batch_size, 1))
        g_loss = self._dcgan.train_on_batch(random_latent_vectors, [misleading_targets,misleading_targets])
        return d1_loss,d2_loss, g_loss

    def predict(self, latent_vector):
        return self._generator.predict(latent_vector)

    def load_weights(self, file_path, by_name=False):
        self._dcgan.load_weights(file_path, by_name)

    def save_weights(self, file_path, overwrite=True):
        self._dcgan.save_weights(file_path, overwrite)

Using TensorFlow backend.


In [2]:
import os
import numpy as np
import keras
from keras.preprocessing import image
from PIL import Image

# Normalize image from 0 - 255 to -1 - 1
def normalize(X):
    return (X - 127.5)/127.5

# Denormalize from -1 - 1 to 0 - 255
def denormalize(X):
    return (X + 1.0)*127.5

def train(latent_dim, height, width, channels,file_path,  X1_train, Y1_train, X2_train, Y2_train): 
    X1_train = X1_train.reshape((X1_train.shape[0],) + (height, width, channels)).astype('float32')
    X1_train = normalize(X1_train)
    
    X2_train = X2_train.reshape((X2_train.shape[0],) + (height, width, channels)).astype('float32')
    X2_train = normalize(X2_train)

    epochs = 40
    batch_size = 128
    iterations = X1_train.shape[0]//batch_size
    dcgan = DCGAN(latent_dim, height, width, channels)
    #dcgan.load_weights(file_path+'gan_epoch20.h5')
    
    for epoch in range(epochs):
        for iteration in range(iterations):
            real_images = X1_train[iteration*batch_size:(iteration+1)*batch_size]
            d1_loss,d2_loss,  g_loss = dcgan.train(real_images, batch_size)
            if (iteration + 1)%10 == 0:
                print('discriminator loss:', d1_loss,d2_loss)
                print('generator loss:', g_loss)
                print(iteration)
                with open(file_path+'loss.txt', 'a') as f:
                    f.write(str(d1_loss) + ',' +str(d2_loss) + ',' + str(g_loss) + '\r')
        dcgan.save_weights(file_path+'gan' + '_epoch' + str(epoch + 1) + '.h5')
        print('epoch' + str(epoch) + ' end')
        print()

def predict(latent_dim, height, width, channels,file_path, epoch):
    np.random.seed(seed=32)
    random_latent_vectors = np.random.normal(size=(100, latent_dim))
    dcgan = DCGAN(latent_dim, height, width, channels)
    dcgan.load_weights(file_path+'gan_epoch'+str(epoch)+'.h5')
    generated_images = dcgan.predict(random_latent_vectors)
    
    for i, generated_image in enumerate(generated_images):
        img = image.array_to_img(denormalize(generated_image), scale=False)
        img.save(os.path.join(file_path+ str(i) + '.png'))
        
    canvas = Image.new('RGB', ((width+1)*10, (height+1)*10), (255, 255, 255))
    for i in range(100):
        xpng = Image.open(file_path+ str(i) + '.png', 'r')
        canvas.paste(xpng, (i%10*(width+1), i//10*(height+1)))
    canvas.save(file_path+'images'+str(epoch)+'.jpg', 'JPEG', quality=100, optimize=True)

In [4]:
latent_dim = 100
height = 28
width = 28
channels = 1
file_path = "/root/userspace/private/gan/generated_mnist_twozero5000/"


In [8]:
(cifar_X , cifar_y), (_, _) = keras.datasets.mnist.load_data()
X1_train=[]
Y1_train=[]
X2_train=[]
Y2_train=[]
for x in range(cifar_X.shape[0]):
    if cifar_y[x]==2:
        X1_train.append(cifar_X[x])
        Y1_train.append(cifar_y[x])
    if cifar_y[x]==0:
        X2_train.append(cifar_X[x])
        Y2_train.append(cifar_y[x])
X1_train=np.array(X1_train[:5000])
Y1_train=np.array(Y1_train[:5000])
X2_train=np.array(X2_train[:5000])
Y2_train=np.array(Y2_train[:5000])

print(X1_train.shape)

(5000, 28, 28)


In [9]:
train(latent_dim, height, width, channels, file_path, X1_train, Y1_train, X2_train, Y2_train)

discriminator loss: 0.450358301401 0.370518416166
generator loss: [1.4972458, 0.61946827, 0.87777746]
9
discriminator loss: 0.390571534634 0.283288478851
generator loss: [2.2389231, 0.96641374, 1.2725093]
19
discriminator loss: 0.398700654507 0.195259064436
generator loss: [3.1274071, 1.2266022, 1.9008048]
29
epoch0 end

discriminator loss: 0.256847977638 0.0619458556175
generator loss: [5.4708905, 1.5564638, 3.9144266]
9
discriminator loss: 0.16931578517 0.0323775708675
generator loss: [5.9722786, 1.7386997, 4.2335787]
19
discriminator loss: 0.130286887288 0.0933616086841
generator loss: [5.5966091, 2.2533116, 3.3432975]
29
epoch1 end

discriminator loss: 0.00540664419532 0.0591967925429
generator loss: [9.3263664, 5.2609448, 4.0654221]
9
discriminator loss: 0.00149547308683 0.00722903758287
generator loss: [11.581972, 5.6229343, 5.9590373]
19
discriminator loss: 0.033609226346 0.00461938977242
generator loss: [9.8441277, 4.2312441, 5.6128831]
29
epoch2 end

discriminator loss: 0.0934

In [5]:
for i in range(10,41,1):
    predict(latent_dim, height, width, channels,file_path, epoch=i)